In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

from base64 import b64decode

def decode_base64(data):
    missing_padding = len(data) % 4
    if missing_padding != 0:
        data += '='* (4 - missing_padding)
    return int.from_bytes(b64decode(data), 'big')

In [3]:
# VAMOS UTILIZAR O decode_base64() PARA TRANSFORMAR O product_id
# EM UM INTEIRO (hexidecimal) PARA UTILIZAR NO MODELO DE MACHINE LEARNING

# por desencargo de consciência, vamos verificar se a função está correta
print(decode_base64('bea127ac-db11-4671-b988-11046e2d2961') == 2694769887921308754649648236518067152934926705487553224373)
print(decode_base64('bea127ac-db11-4671-b988-11046e2d2961'))
print(decode_base64('bea127ac-db11-4671-b988-11046e2d2962'))
print(decode_base64('bea127ac-db11-4671-b988-slgkasgskalgmas;lgam,'))

True
2694769887921308754649648236518067152934926705487553224373
2694769887921308754649648236518067152934926705487553224374
2962930825950107269821114141119787861005907558195596562885061774770601


In [4]:
data = pd.read_csv('product_entries.csv')

data = data.drop(columns=['name'])

print(data.head(5))  

# Agrupar por 'name' e somar as quantidades
grouped_data = data.groupby('product_id')['quantity'].sum().reset_index()

                             product_id  quantity             entry_at
0  bea127ac-db11-4671-b988-11046e2d2961      2033  2024-03-18 03:43:53
1  bea127ac-db11-4671-b988-11046e2d2961      2226  2024-03-16 04:38:12
2  bea127ac-db11-4671-b988-11046e2d2961      1697  2024-02-22 05:03:53
3  bea127ac-db11-4671-b988-11046e2d2961      2900  2024-04-17 10:03:26
4  bea127ac-db11-4671-b988-11046e2d2961      1978  2024-03-01 02:56:35


In [5]:
# Converter a coluna 'entry_at' para datetime
data['entry_at'] = pd.to_datetime(data['entry_at'])

# Extrair características relevantes da data, como ano, mês, dia e hora
data['year'] = data['entry_at'].dt.year
data['month'] = data['entry_at'].dt.month
data['day'] = data['entry_at'].dt.day
data['hour'] = data['entry_at'].dt.hour

# Dropar as colunas que não serão usadas
data = data.drop(columns=['entry_at'])

# Exibir os primeiros registros para verificar as transformações
print(data.head())

                             product_id  quantity  year  month  day  hour
0  bea127ac-db11-4671-b988-11046e2d2961      2033  2024      3   18     3
1  bea127ac-db11-4671-b988-11046e2d2961      2226  2024      3   16     4
2  bea127ac-db11-4671-b988-11046e2d2961      1697  2024      2   22     5
3  bea127ac-db11-4671-b988-11046e2d2961      2900  2024      4   17    10
4  bea127ac-db11-4671-b988-11046e2d2961      1978  2024      3    1     2


In [6]:
data['product_id'] = data['product_id'].apply(decode_base64)

# Definir as variáveis dependente (y) e independentes (X)
X = data.drop(columns=['quantity'])
y = data['quantity']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Inicializar o modelo
lr = LinearRegression()

# Treinar o modelo
lr.fit(X_train, y_train)

# Fazer previsões
y_pred_lr = lr.predict(X_test)

# Calcular as métricas
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print(f"Linear Regression RMSE: {rmse_lr}, R²: {r2_lr}")

Linear Regression RMSE: 882.41648825516, R²: -0.016927187419655043


In [13]:
# Suponha que temos um novo produto com as seguintes características
novo_produto = pd.DataFrame({
    'product_id': [decode_base64('bea127ac-db11-4671-b988-11046e2d2961')],
    'year': [2024],
    'month': [3],
    'day': [18],
    'hour': [3]
})


# Precisamos preparar o novo produto da mesma maneira que preparamos os dados de treinamento
# Isso pode incluir a conversão de 'entry_at' para um formato de data e hora e a codificação de 'product_id'

# Usamos o modelo para prever a quantidade
quantidade_prevista = lr.predict(novo_produto)

print(f"A quantidade prevista para o novo produto é: {quantidade_prevista}")

A quantidade prevista para o novo produto é: [1535.74750947]
